In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lead, col, broadcast, collect_set, size, array_contains

class Transformer:

    def __init__(self):
        pass

    def transform(self, inputDFs):
        pass

class FirstTransformer(Transformer):

    def transform(self, inputDFs):
        #Find Customed who have bought ipads after iphone
        transactionInputDF = inputDFs.get("transactionInputDF")

        print("transactionInputDF in transform")
        transactionInputDF.show()

        windowspec = Window.partitionBy("customer_id").orderBy("transaction_date")

        transformedDF = transactionInputDF.withColumn(
            "next_product_name", lead("product_name").over(windowspec)
            )
        print("airpods after buying iphones")
        transformedDF.orderBy("customer_id", "transaction_date", "product_name").show()

        filteredDF = transformedDF.filter(
            (col("product_name") == "iPhone") & (col("next_product_name") == "AirPods")
        )

        filteredDF.orderBy("customer_id", "transaction_date", "product_name").show()

        customerInputDF = inputDFs.get("customerInputDF")
        
        customerInputDF.show()

        joinDF = customerInputDF.join(
            broadcast(filteredDF),
            "customer_id"
        )
        print("joined DF")
        
        joinDF.show()

        return joinDF.select(
            "customer_id",
            "customer_name",
            "location"
        )

class SecondTransformer(Transformer):

    def transform(self, inputDFs):
        ''' customer who have bought Iphones and Airpods
        '''
        transactionInputDF = inputDFs.get("transactionInputDF")

        print("transactionInputDF in transform")

        groupDF = transactionInputDF.groupBy("customer_id").agg(
            collect_set("product_name").alias("products")
        )
        print("groupedDF")
        groupDF.show()

        filteredDF = groupDF.filter(
            (array_contains(col("products"),"iPhone")) & 
            (array_contains(col("products"), "AirPods")) &
            (size(col("products")) == 2)
        )
        print("only Airpod And Iphone")
        filteredDF.show()

        customerInputDF = inputDFs.get("customerInputDF")
        
        customerInputDF.show()

        joinDF = customerInputDF.join(
            broadcast(filteredDF),
            "customer_id"
        )
        print("joined DF")
        
        joinDF.show()

        return joinDF.select(
            "customer_id",
            "customer_name",
            "location"
        )
